In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision.datasets import cifar
from torch.utils.data import DataLoader
from torchvision.models import resnet50

In [ ]:
class UpsampleBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(UpsampleBlock, self).__init__()
        self.upsample = nn.Sequential(
            nn.ConvTranspose2d(in_channel, out_channel, kernel_size=2, stride=2),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
        )

    def forward(self, x):
        return self.upsample(x)

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(*list(resnet50().children())[:-2])

    def forward(self, x):
        return self.encoder(x)

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            UpsampleBlock(2048, 1024),
            UpsampleBlock(1024, 512),
            UpsampleBlock(512, 256),
            UpsampleBlock(256, 64),
            UpsampleBlock(64, 3)
        )

    def forward(self, x):
        return self.decoder(x)

In [ ]:
class EncoderDecoder(nn.Module):
    def __init__(self):
        super(EncoderDecoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [ ]:
train_data = cifar.CIFAR100(root='./data', train=True, transform=transforms, download=True)
test_data = cifar.CIFAR100(root='./data', train=False, transform=transforms, download=True)

In [ ]:
batch_size = 8
learning_rate = 0.01
num_epochs = 10

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
model = EncoderDecoder().to(device)

In [ ]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    train_loss, val_loss = [], []
    
    model.train()
    for image, _ in train_loader:
        image = image.to(device)
        
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, image)
        train_loss.append(loss.item())

        loss.backward()
        optimizer.step()

    model.eval()
    for image, _ in test_loader:
        image = image.to(device)
        
        outputs = model(image)
        loss = criterion(outputs, image)
        val_loss.append(loss.item())

    print(f'Epoch: {epoch} \t Train Loss: {train_loss[-1]} \t Val Loss: {val_loss[-1]}')

In [1]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(train_loss, label='training loss', color='red')
plt.plot(val_loss, label='validation loss', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.show()